<a href="https://colab.research.google.com/github/Chirag314/decisiontrees-backorders/blob/main/Decision_trees_backorders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###This notebook is copied from exercises from book Ensemble Machine Learning Cookbook.

In [5]:
pip install --upgrade scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
#import required libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import  roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

import itertools
from sklearn import tree

In [10]:
# Read data from github. Use raw format and copy url# Note normal url and raw url will be different.
import pandas as pd
pd.options.display.max_rows=None
pd.options.display.max_columns=None
url = 'https://raw.githubusercontent.com/PacktPublishing/Ensemble-Machine-Learning-Cookbook/master/Chapter04/Decision%20Tree/backorders.csv'
df_backorder = pd.read_csv(url)
#df = pd.read_csv(url)
print(df_backorder.head(5))

       sku  national_inv  lead_time  in_transit_qty  forecast_3_month  \
0  1888279           117        NaN               0                 0   
1  1870557             7        2.0               0                 0   
2  1475481           258       15.0              10                10   
3  1758220            46        2.0               0                 0   
4  1360312             2        2.0               0                 4   

   forecast_6_month  forecast_9_month  sales_1_month  sales_3_month  \
0                 0                 0              0              0   
1                 0                 0              0              0   
2                77               184             46            132   
3                 0                 0              1              2   
4                 6                10              2              2   

   sales_6_month  sales_9_month  min_bank potential_issue  pieces_past_due  \
0             15             15         1              N

In [15]:
df_backorder.shape

(61589, 22)

In [11]:
df_backorder.shape
df_backorder.describe()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty
count,6.158900e+04,61589.000000,58186.000000,61589.000000,6.158900e+04,6.158900e+04,6.158900e+04,61589.000000,61589.000000,6.158900e+04,6.158900e+04,61589.000000,61589.000000,61589.000000,61589.000000,61589.000000
mean,2.037188e+06,287.721882,7.559619,30.192843,1.692728e+02,3.150413e+02,4.535760e+02,44.742957,150.732631,2.835465e+02,4.196427e+02,43.087256,1.605400,-6.264182,-5.863664,1.205361
std,6.564178e+05,4233.906931,6.498952,792.869253,5.286742e+03,9.774362e+03,1.420201e+04,1373.805831,5224.959649,8.872270e+03,1.269858e+04,959.614135,42.309229,25.537906,24.844514,29.981155
min,1.068628e+06,-2999.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,-99.000000,-99.000000,0.000000
25%,1.498574e+06,3.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.620000,0.640000,0.000000
50%,1.898033e+06,10.000000,8.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2.000000,4.000000e+00,6.000000e+00,0.000000,0.000000,0.820000,0.800000,0.000000
75%,2.314826e+06,57.000000,8.000000,0.000000,1.200000e+01,2.500000e+01,3.600000e+01,6.000000,17.000000,3.400000e+01,5.100000e+01,3.000000,0.000000,0.960000,0.950000,0.000000
max,3.284895e+06,673445.000000,52.000000,170976.000000,1.126656e+06,2.094336e+06,3.062016e+06,295197.000000,934593.000000,1.799099e+06,2.631590e+06,192978.000000,7392.000000,1.000000,1.000000,2999.000000


In [12]:
# Check the data types
df_backorder.dtypes

sku                    int64
national_inv           int64
lead_time            float64
in_transit_qty         int64
forecast_3_month       int64
forecast_6_month       int64
forecast_9_month       int64
sales_1_month          int64
sales_3_month          int64
sales_6_month          int64
sales_9_month          int64
min_bank               int64
potential_issue       object
pieces_past_due        int64
perf_6_month_avg     float64
perf_12_month_avg    float64
local_bo_qty           int64
deck_risk             object
oe_constraint         object
ppap_risk             object
stop_auto_buy         object
rev_stop              object
went_on_backorder     object
dtype: object

We can see that sku is an identifier and will be of no use to us for our model-building exercise. We will, therefore, drop sku from our DataFrame as follows:

In [13]:
df_backorder.drop('sku',axis=1,inplace=True)

In [14]:
# check missing values
df_backorder.isnull().sum()

national_inv            0
lead_time            3403
in_transit_qty          0
forecast_3_month        0
forecast_6_month        0
forecast_9_month        0
sales_1_month           0
sales_3_month           0
sales_6_month           0
sales_9_month           0
min_bank                0
potential_issue         0
pieces_past_due         0
perf_6_month_avg        0
perf_12_month_avg       0
local_bo_qty            0
deck_risk               0
oe_constraint           0
ppap_risk               0
stop_auto_buy           0
rev_stop                0
went_on_backorder       0
dtype: int64

#####since number of missing values in the lead_time variable is about 5% , we will remove all the observaions where lead_time is missing for our analysis

In [16]:
df_backorder=df_backorder.dropna(axis=0)

In [21]:
# We now need to encode our categorical variables. We select only the categorical variables and call pd.get_dummies() to dummy-code the non-numeric variables:
non_numeric_attributes=df_backorder.select_dtypes(include=['object']).columns
df_backorder=pd.get_dummies(columns=non_numeric_attributes,data=df_backorder,prefix=non_numeric_attributes, prefix_sep='_',drop_first=True)
df_backorder.dtypes

national_inv               int64
lead_time                float64
in_transit_qty             int64
forecast_3_month           int64
forecast_6_month           int64
forecast_9_month           int64
sales_1_month              int64
sales_3_month              int64
sales_6_month              int64
sales_9_month              int64
min_bank                   int64
pieces_past_due            int64
perf_6_month_avg         float64
perf_12_month_avg        float64
local_bo_qty               int64
potential_issue-Yes        uint8
deck_risk-Yes              uint8
oe_constraint-Yes          uint8
ppap_risk-Yes              uint8
stop_auto_buy-Yes          uint8
rev_stop-Yes               uint8
went_on_backorder-Yes      uint8
dtype: object

With the preceding code, we get to see the datatypes. We notice that dummy-coded variables are all of the unsigned integer (uint8) type:

In [22]:
#We will then look at our target variable distribution as follows:
pd.value_counts(df_backorder['went_on_backorder-Yes'].values)


0    47217
1    10969
dtype: int64

In [23]:
# We will now split our data into training and testing datasets:
#Performing train test split on the data
X,Y=df_backorder.loc[:,df_backorder.columns!='went_on_backorder-Yes'].values,df_backorder.loc[:,'went_on_backorder-Yes'].values

# Split our dataset into train & test set
X_train, X_test, Y_train, Y_test=train_test_split(X, Y, test_size=0.2,random_state=1)

In [24]:
#We can use the model to predict our class labels using both our training and our testing datasets:
classifier=tree.DecisionTreeClassifier(random_state=1)
#Fit our model to the data
model_DT_Gini=classifier.fit(X_train, Y_train)
model_DT_Gini

DecisionTreeClassifier(random_state=1)

In [25]:
test_predictedvaues=model_DT_Gini.predict(X_test)

#Check accuracy
acc=accuracy_score(Y_test,test_predictedvaues)
print("Accuracy is ",acc)

# CHeck TP TN FP FN
tp,fp, fn,tn=confusion_matrix(Y_test, test_predictedvaues).ravel()
print("TN:",tn, " FP:",fp, " FN:",fn, " TP:",tp)

Accuracy is  0.9009279945007733
TN: 1701  FP: 655  FN: 498  TP: 8784


In [50]:

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
#We will now use a plot_confusion_matrix function to plot our confusion matrix.
# This function is taken from http://scikit-learn.org and is readily available there, so we won't show this function here. It is, however, provided with the code in the book for your reference
target_names=["No","Yes"]
# pass actual and predicted values to confusin matrix
cm=confusion_matrix(Y_test, test_predictedvaues)
plt.figure()
plot_confusion_matrix(cm,Y_test, test_predictedvaues, labels=target_names, normalize=False)
plt.show()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator.
  warnings.warn(msg, category=FutureWarning)


ValueError: ignored

<Figure size 432x288 with 0 Axes>

In [41]:
#We can change the hyperparameters to tune our model. 
#We can also perform a grid search to find the hyperparameter values that supply optimum results. We can use the following code to set the hyperparameter values:
grid_search_parameters={'criterion':['gini','entropy'],
                        'min_samples_split':[2],
                        'max_depth':[None,2,3],
                        'min_samples_leaf':[1,5],
                        'max_leaf_nodes':[None],}

In [42]:
#We will use GridSearchCV() to grid search the parameters:
from sklearn.model_selection import GridSearchCV
classifier=tree.DecisionTreeClassifier()

# Use Grisearchcv pass the values
model_DT_Grid=GridSearchCV(classifier,grid_search_parameters,cv=10)
model_DT_Grid.fit(X_train, Y_train)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 2, 3], 'max_leaf_nodes': [None],
                         'min_samples_leaf': [1, 5], 'min_samples_split': [2]})

In [43]:
#After running the preceding command, we can see the best parameter values among those provided using best_params_: #
model_DT_Grid.best_params_

{'criterion': 'entropy',
 'max_depth': None,
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [44]:
#use the model that is selected using the GridSearchCV() function:
test_predictedvaues=model_DT_Grid.predict(X_test)
acc=accuracy_score(Y_test,test_predictedvaues)
print("Accuracy is :",acc)
tn, fp, fn, tp = confusion_matrix(Y_test, test_predictedvaues).ravel()
print("TN:",tn, " FP:",fp, " FN:",fn, " TP:",tp)

cm = confusion_matrix(Y_test, test_predictedvaues)

plt.figure()
plot_confusion_matrix(cm, classes=target_names, normalize=False)
plt.show()


Accuracy is : 0.9025605774188005
TN: 8799  FP: 640  FN: 494  TP: 1705


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator.
  warnings.warn(msg, category=FutureWarning)


TypeError: ignored

<Figure size 432x288 with 0 Axes>

In [45]:
from sklearn.metrics import classification_report
#In order to see the metrics per-label, we can also use the classification_report, as follows:
print(classification_report(Y_test,test_predictedvaues,target_names=target_names))

              precision    recall  f1-score   support

          No       0.95      0.93      0.94      9439
         Yes       0.73      0.78      0.75      2199

    accuracy                           0.90     11638
   macro avg       0.84      0.85      0.84     11638
weighted avg       0.91      0.90      0.90     11638

